# Clean Up Annotations from OMERO Project

This notebook demonstrates how to clean up annotations from an OMERO project using the omero-annotate-ai package.

In [14]:
import omero_annotate_ai
from omero_annotate_ai import create_omero_connection_widget
from omero_annotate_ai.omero.omero_functions import list_annotation_tables
from omero_annotate_ai.omero.omero_utils import delete_table

## Step 1: Connect to OMERO

First, establish a connection to your OMERO server.

In [ ]:
# Connect to OMERO server
conn_widget = create_omero_connection_widget()
conn_widget.display()

# Get the connection object
conn = conn_widget.get_connection()

📄 Loaded configuration from connection history: root@localhost
⚠️ Error loading password from keychain: No recommended backend was available. Install a recommended 3rd party backend package; or, install the keyrings.alt package if you want to use the non-recommended backends. See https://pypi.org/project/keyring for details.


## Step 2: Find Available Projects/Datasets

First, let's see what projects and datasets are available to work with.

In [16]:
import ezomero

# List available projects
print("Available projects:")
projects = ezomero.get_projects(conn)
for project in projects:
    print(f"  - Project ID: {project['id']}, Name: {project['name']}")

print("\nAvailable datasets:")
datasets = ezomero.get_datasets(conn)
for dataset in datasets:
    print(f"  - Dataset ID: {dataset['id']}, Name: {dataset['name']}")

Available projects:


AttributeError: module 'ezomero' has no attribute 'get_projects'

## Step 3: List Annotation Tables

View all annotation tables in your chosen container.

In [ ]:
# Specify your container type and ID (from the list above)
container_type = "project"  # or "dataset"
container_id = 123  # Replace with actual ID from the list above

# List all annotation tables in the container
tables = list_annotation_tables(conn, container_type, container_id)
print(f"Found {len(tables)} annotation tables in {container_type} {container_id}:")

for table in tables:
    print(f"  - Table ID: {table['id']}, Name: {table['name']}")
    if 'progress_percent' in table:
        print(f"    Progress: {table['progress_percent']:.1f}% complete ({table['completed_units']}/{table['total_units']} units)")

## Step 4: Delete Annotation Tables

Remove specific annotation tables by their ID.

In [ ]:
# Delete specific annotation table by ID
table_id_to_delete = 102  # Replace with the actual table ID you want to delete

# Confirm deletion
print(f"Deleting annotation table with ID: {table_id_to_delete}")
success = delete_table(conn, table_id_to_delete)

if success:
    print("✅ Annotation table deleted successfully")
else:
    print("❌ Failed to delete annotation table")

Deleting annotation table with ID: 102
❌ Error deleting table 102: 'NoneType' object has no attribute 'getObject'
❌ Failed to delete annotation table


-! 07/18/25 00:26:23.863 --Ice.Config=: warning: Proxy keep alive failed.
-! 07/18/25 00:26:23.888 --Ice.Config=: warning: Proxy keep alive failed.
-! 07/18/25 00:26:23.897 --Ice.Config=: warning: Proxy keep alive failed.
-! 07/18/25 00:26:23.898 --Ice.Config=: warning: Proxy keep alive failed.
-! 07/18/25 00:26:23.899 --Ice.Config=: warning: Proxy keep alive failed.
-! 07/18/25 00:26:23.917 --Ice.Config=: warning: Proxy keep alive failed.


## Step 5: Bulk Cleanup (Optional)

For cleaning up multiple annotation tables at once.

In [ ]:
# Bulk delete multiple annotation tables
table_ids_to_delete = [456, 789, 101]  # Replace with actual table IDs

print(f"Deleting {len(table_ids_to_delete)} annotation tables...")
deleted_count = 0

for table_id in table_ids_to_delete:
    success = delete_table(conn, table_id)
    if success:
        print(f"✅ Deleted table {table_id}")
        deleted_count += 1
    else:
        print(f"❌ Failed to delete table {table_id}")

print(f"\nCleanup complete: {deleted_count}/{len(table_ids_to_delete)} tables deleted")

## Step 6: Complete Project Cleanup (Optional)

Use the cleanup function to remove all omero-annotate-ai annotations from a project.

In [ ]:
from omero_annotate_ai.omero.omero_functions import cleanup_project_annotations

# Complete project cleanup - removes all annotation tables, ROIs, and map annotations
project_id = 123  # Replace with your actual project ID

# Optional: specify training set name to only clean up specific annotations
# trainingset_name = "my_training_set"  # Uncomment and set if needed
trainingset_name = None  # Clean up all annotations

print(f"🧹 Starting cleanup of project {project_id}...")
results = cleanup_project_annotations(conn, project_id, trainingset_name)

print(f"\n📊 Cleanup Results:")
print(f"   Tables deleted: {results['tables']}")
print(f"   ROIs deleted: {results['rois']}")
print(f"   Map annotations deleted: {results['map_annotations']}")
print(f"   Images processed: {results['images_processed']}")

In [ ]:
# Close the connection when done
if conn:
    conn.close()
    print("Connection closed")